<a href="https://colab.research.google.com/github/distractedm1nd/mar2moon/blob/main/scripts/Wav2Vec2_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install soundfile
!pip install jiwer

In [ ]:
from datasets import Dataset
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import re
import os
import random

#df = pd.read_excel("../data/Sentiment_Labelling_Altcoin_Daily_210401_to_210531_Batch_1.xlsx")
df = pd.read_csv("data2.csv", sep=";")
df["file"] = df.apply(lambda x: f"drive/MyDrive/clips/2/{x['Podcast_Title'][:-7]}_{x.name}.wav" if f"{x['Podcast_Title'][:-7]}_{x.name}.wav" in os.listdir("drive/MyDrive/clips/2") else np.nan, axis=1)
batch = Dataset.from_pandas(df.dropna(subset=["file"]).head(154)).shuffle().train_test_split(test_size=0.2)
df.dropna(subset=["file"]).head(154)

,Podcast_Title,Start_Time,End_Time,Auto_Label,Text,Coin,Sentiment,file,audio,predicted_text
0,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,00:00:20.080,00:00:28.160,BTC,od outperform bitcoin and is watching it very ...,BTC,bearish,drive/MyDrive/clips/2/Altcoin Daily_20210401_E...,[-0.065979 -0.08096313 -0.10205078 ... -0.02...,OD OUTPERFORM BITCOIN AND IS WATCHING IT VERY ...
1,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,00:01:25.680,00:01:36.799,BTC,s a slightly more nuanced opinion than many bi...,BTC,bearish,drive/MyDrive/clips/2/Altcoin Daily_20210401_E...,[-0.01321411 -0.00866699 -0.00341797 ... 0.00...,S A SLIGHTLY MORE NUANCED OPINION THAN MANY BI...
2,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,00:02:00.719,00:02:10.239,ETH,be more um sound from a monetary perspective t...,ETH,bullish,drive/MyDrive/clips/2/Altcoin Daily_20210401_E...,[0. 0. 0. ... 0.000274...,MORE AM SOUND FROM A MONETARY PERSPECTIVE AN B...
3,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,00:02:28.560,00:02:34.400,ETH,and bitcoin hard forks are extremely rare in e...,ETH,bearish,drive/MyDrive/clips/2/Altcoin Daily_20210401_E...,[3.05175781e-05 0.00000000e+00 1.22070312e-04 ...,AND BECO AND HARDFORK'SARE EXTREMELY RARE IN A...
4,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,00:03:47.840,00:03:56.239,ETH,but nevertheless the minority a hundred percen...,ETH,bullish,drive/MyDrive/clips/2/Altcoin Daily_20210401_E...,[-0.00460815 -0.00427246 -0.00384521 ... -0.00...,BUT NEVERTHELESS THE MINORITY AA HUNDRED PER C...
...,...,...,...,...,...,...,...,...,...,...
168,Altcoin Daily_20210423_URGENT! Cryptocurrency ...,00:11:03.200,00:11:15.200,ETH,ing to do big things uniswap has deployed thes...,ETH,NaN,drive/MyDrive/clips/2/Altcoin Daily_20210423_U...,[-0.01065063 -0.01156616 -0.01086426 ... 0.00...,ING TO DO BIG THINGS UNISWAP HAS DEPLOYED THES...
169,Altcoin Daily_20210423_URGENT! Cryptocurrency ...,00:11:55.360,00:12:03.839,None,don't you think by the way google search volum...,NaN,NaN,drive/MyDrive/clips/2/Altcoin Daily_20210423_U...,[0.00039673 0.00039673 0.0005188 ... 0.063110...,DON'T YOU THINK BY THE WAY GOUGLE SEARCH VOLUM...
170,Altcoin Daily_20210424_'NFTs are BIGGEST OPPOR...,00:03:03.599,00:03:09.200,crypto_space,point in crypto and blockchain so yes i'm in l...,NaN,NaN,drive/MyDrive/clips/2/Altcoin Daily_20210424_'...,[ 0.01144409 0.01184082 -0.00665283 ... 0.00...,POINT CRIPTO AND BLOCK CHAIN SO YES I'M IN LOV...
171,Altcoin Daily_20210424_'NFTs are BIGGEST OPPOR...,00:05:19.280,00:05:27.440,crypto_space,like people who don't collect nfts have no cry...,NaN,NaN,drive/MyDrive/clips/2/Altcoin Daily_20210424_'...,[-0.04092407 -0.05355835 -0.05679321 ... 0.00...,IKE PEOPLE WHO DON'T COLLECT ENOFTEES HAVE NO ...


In [ ]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\d\[\]]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["Text"]).upper()
    return batch

batch = batch.map(remove_special_characters)

def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = batch.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=batch.column_names["train"])

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 10,
 "'": 16,
 'A': 27,
 'B': 19,
 'C': 25,
 'D': 28,
 'E': 2,
 'F': 0,
 'G': 5,
 'H': 21,
 'I': 18,
 'J': 6,
 'K': 14,
 'L': 7,
 'M': 26,
 'N': 8,
 'O': 20,
 'P': 13,
 'Q': 15,
 'R': 23,
 'S': 9,
 'T': 12,
 'U': 22,
 'V': 1,
 'W': 17,
 'X': 4,
 'Y': 3,
 'Z': 11,
 '\xa0': 24}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

31


In [ ]:
import json
with open('vocab2.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab2.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

#processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
#processor.save_pretrained("wav2vec2-base-processor")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")

In [ ]:
import soundfile as sf

total_audiolength = 0
def speech_file_to_array_fn(batch):
    global total_audiolength
    speech_array, sampling_rate = sf.read(batch["file"])
    total_audiolength += len(speech_array)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

batch = batch.map(speech_file_to_array_fn, remove_columns=batch.column_names["train"], num_proc=4)
print(total_audiolength)





209904


In [ ]:
import IPython.display as ipd
import numpy as np

print(batch["train"][5]["target_text"])
ipd.Audio(data=np.asarray(batch["train"][5]["speech"]), autoplay=False, rate=16000)

IVE ANDREAS DIETRICH WHO HELPS LEAD THE COMPANY'S BLOCKCHAIN INITIATIVE DEUTSCHE TELEKOM SEES A VARIETY OF OPPORTUNITIES USING CELLO INCLU


In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

batch_prepared = batch.map(prepare_dataset, remove_columns=batch.column_names["train"], num_proc=4, batched=True, batch_size=8)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
import torch
import numpy as np
from datasets import load_metric
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    print(pred_str)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-960h",
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import Trainer
from transformers import TrainingArguments
training_args = TrainingArguments(
  output_dir="wav2vec2-test",
  group_by_length=True,
  per_device_train_batch_size=4,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=False,
  learning_rate=1e-5,
  weight_decay=0.005,
  save_steps=50,
  eval_steps=50,
  logging_steps=50,
  warmup_steps=250,
  save_total_limit=2,
)


trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=batch_prepared["train"],
    eval_dataset=batch_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
50,1.061500,0.670797,0.277037,6.029100,5.142000
100,0.843200,0.624291,0.287407,5.980700,5.183000
150,0.762600,0.567986,0.280000,6.054200,5.120000
200,0.627100,0.569063,0.248889,6.008000,5.160000
250,0.586200,0.505535,0.234074,5.999600,5.167000
300,0.530100,0.472824,0.223704,6.014700,5.154000
350,0.472300,0.507768,0.216296,6.000000,5.167000
400,0.472500,0.548698,0.211852,5.993700,5.172000
450,0.422100,0.517611,0.211852,6.039800,5.133000
500,0.414300,0.518192,0.189630,6.091300,5.089000


['PROJECTS TO POCODOT INTERESTING SO THIS SUMS IT UP IN A NUTSHELL GIMMYOR THOUGHTS', "S A SLIGHTLY MORE NUANCED OPINION THAN MANY BITCOIN MAXIMALISTS IN THE SPACE LISTEN TO THIS WHAT ABOUT BITGUAN'S COMPARISON TO OTHER CURRENT CRYPTIC CURRENCIES SO BIT", 'THE SORT OF AH TO FIFTY CAPE LUS RANGE I THINK THAT PUTS ETHERIUM AM AT A TWENTY', 'ORIN A BIT COIGN AT FIRST THERE WAS A CHANCE HE WOULD NEVER WARM UP TO EXARPEE AND START PUSHING ETH AL OVER THE PLACE RIGHT AND', 'LIKE NASPERS VENTURES GALOXY DIGIL AND APEX CAPITAL PARTNERS MUTABLE X IS THE FIRST LAYER TO SCALING SOLUTION FOR AN OF TET', "A ONE MILLION BIT COIN BY TWENTY TWENTY FIVE SOUNDS REASONABLE DO YOU AGREE WITH THIS DO YOU DISAGREE WITH THIS I TEND TO THINK IT'S POSSIBLE NOW", "N YOU LOOK AT GRAFFLI A GRAFFLING G L QU IT'S A NO CODE BLOCKCHAIN BOT BUILD OU'RE REALLY INTERESTING THANKS FOR YOUR NICE WORK THANK YOU MICHAEL I'M SOMEBODY SAD", 'SE OTHER PROTOCOLES ARE A LOT YOUNGER ESPECIALLY YOU KNOTE THE LOWER CAP COINS ETHEOR

TrainOutput(global_step=930, training_loss=0.4888246782364384, metrics={'train_runtime': 1924.691, 'train_samples_per_second': 0.483, 'total_flos': 1.0487960845726925e+18, 'epoch': 30.0, 'init_mem_cpu_alloc_delta': 345986220, 'init_mem_gpu_alloc_delta': 1265227264, 'init_mem_cpu_peaked_delta': 135259892, 'init_mem_gpu_peaked_delta': 3973518848, 'train_mem_cpu_alloc_delta': 441443, 'train_mem_gpu_alloc_delta': 3793746944, 'train_mem_cpu_peaked_delta': 47285142, 'train_mem_gpu_peaked_delta': 2049887744})

In [ ]:
def map_to_result(batch):
    model.to("cuda")
    input_values = processor(
        batch["speech"],
        sampling_rate=batch["sampling_rate"],
        return_tensors="pt"
    ).input_values.to("cuda")

    with torch.no_grad():
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]

    return batch

results = batch.map(map_to_result)

/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
